In [89]:
import yaml
import argparse
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from model_simple import RNN
import pandas  as pd
from providers import MolecularNotationDataset, SpecialTokenWrapperModel, SmilesModel, robust_standardizer
from legogram import LegoGram
from legogram.apps import LegoGramRNNSampler
from utils import collect, unique_func
import os
from torch.utils.tensorboard import SummaryWriter
import json
from torch.nn import MSELoss
from copy import deepcopy


In [90]:
dataset = torch.load('lg.bin')

In [91]:
logits_prior = torch.load("logits_prior.pk")

In [92]:
dataset.vocsize

2974

In [93]:
logits_prior.shape

torch.Size([100, 2974, 140])

In [94]:
from torch.nn import LogSoftmax

In [16]:
nn = LogSoftmax(dim =2)

In [17]:
logits_prior = nn(logits_prior)

In [19]:
logits_prior.shape

torch.Size([100, 2974, 140])

In [21]:
seqs = torch.load("seqs.pk")

In [24]:
seqs = seqs.detach().cpu().numpy()

In [53]:
dataset.model.

In [28]:
for elem in seqs:
    dataset.model.decode(elem)

IndexError: list index out of range

In [ ]:
def sample(self, batch_size,lexical_model, max_length=140):

    start_token = Variable(torch.zeros(batch_size).long())
    start_token[:] = 1 #FixMe -- possible mistake ???

    x = start_token.cuda().unsqueeze(1)

    finished = torch.zeros(batch_size).byte()
    state = lexical_model.init_state(batch_size)
    hidden = self.initHidden(batch_size)
    loss_f = CrossEntropyLoss(reduction='none')
    #hidden = self.endp2hidden(endp)
    #out = self.forward(x)
    log_probs = Variable(torch.zeros(batch_size)).cuda()
    tokens = start_token.clone().cuda()
    tokens = tokens.unsqueeze(1)
    #print(tokens.shape)
    for step in range(max_length):
        logits,hidden = self.forward_to_sample(x,hidden)
        state,x = lexical_model.sample(state,logits.squeeze(2).cpu().detach().numpy(),0)

        x = Variable(torch.cuda.LongTensor(x)).unsqueeze(1)
        tokens = torch.cat([tokens, x], dim=1)
        #print(tokens.shape)
        #print(x.shape)

        #print(logits.shape)
        ##print(x.shape)
        #print(x.is_cuda)
        #print(logits.is_cuda)
        log_probs += loss_f(logits.squeeze(2), x.squeeze(1))
        #rint(x.shape)
    correct = [Chem.MolToSmiles(graph2mol(g)) for g, finished in zip(*state) if finished]
    return correct, log_probs, tokens

In [51]:
def __init__ (self, lg):
    self.lg = lg
    self.special_tokens = ['<pad>', '<bos>', '<eos>']
    self.nspecs = len(self.special_tokens)
    self.vocsize = self.lg.vocsize+self.nspecs

def encode (self, smi):
    return [c+self.nspecs for c in self.lg.encode(smi)]

def decode (self, code):
    return self.lg.decode([c - self.nspecs for c in code])
    #return self.lg.decode([c-self.nspecs for c in code])

def get_compat_rules (self, graph): # mask only
    rule_mask = self.lg.get_compat_rules(graph, as_mask=True)
    return rule_mask#np.hstack((np.ones((self.nspecs), np.float32), rule_mask))

def init_state (self, batch_size):
    graphs = [None] * batch_size
    finished = [False] * batch_size
    return (graphs, finished)

def sample(self, state, logits, placeholder_idx):
    graphs, finished = state
    batch_len = logits.shape[0]
    logits = logits[:,self.nspecs:]
    current = [None] * batch_len
    for i in range(len(graphs)):
        graph = graphs[i]
        if graph and (not finished[i]):
            mask = self.get_compat_rules(graph)
            if mask.sum() == 0.:
                finished[i] = True
                current[i] = placeholder_idx
                continue

            logits_of_a_rule = logits[i] * mask
            bool_mask = mask.astype(np.bool)
            prob = softmax(logits_of_a_rule[bool_mask])
            current[i] = np.random.choice(np.array(range(logits.shape[-1]))[bool_mask], 1, p=prob)[0]
            if not check_compat(graphs[i], self.lg.rules[current[i]]):
                raise Exception(
                    "Failed to combine rules! {} and {}".format(graphs[i], self.lg.rules[current[i]]))
            graphs[i] = combine_rules(graphs[i], self.lg.rules[current[i]])

        elif (not graph):
            prob = softmax(logits[i])
            current[i] = np.random.choice(range(logits.shape[-1]), 1, p=prob)[0]
            graphs[i] = self.lg.rules[current[i]]
        else:
            current[i] = placeholder_idx
    state = (graphs, finished)
    return state, [c for c in current]



array([[ 206,  756, 2476, ..., 2165,  817, 1469],
       [1142, 1354, 2575, ...,  352,   45, 1065],
       [1425, 2517, 2168, ..., 1101,  127,  115],
       ...,
       [ 515, 1660,  100, ...,  125,   26,   38],
       [1191, 2258, 1294, ..., 1807,  951, 1346],
       [1968,  725, 1485, ...,   33,   62,   29]])

In [36]:
elem = list(seqs[0, :])

In [47]:
dataset.model.encode("c1ccccc1")

[144, 3, 3, 3, 3, 3, 3]

In [50]:
dataset.model.decode([144, 3, 3, 3, 3, 3, 3, 2, 2,2])

IndexError: list index out of range

In [ ]:
for step in range(max_length):
        logits,hidden = self.forward_to_sample(x,hidden)
        state,x = lexical_model.sample(state,logits.squeeze(2).cpu().detach().numpy(),0)

        x = Variable(torch.cuda.LongTensor(x)).unsqueeze(1)
        tokens = torch.cat([tokens, x], dim=1)
        #print(tokens.shape)
        #print(x.shape)

        #print(logits.shape)
        ##print(x.shape)
        #print(x.is_cuda)
        #print(logits.is_cuda)
        log_probs += loss_f(logits.squeeze(2), x.squeeze(1))

In [52]:
seqs

array([[ 206,  756, 2476, ..., 2165,  817, 1469],
       [1142, 1354, 2575, ...,  352,   45, 1065],
       [1425, 2517, 2168, ..., 1101,  127,  115],
       ...,
       [ 515, 1660,  100, ...,  125,   26,   38],
       [1191, 2258, 1294, ..., 1807,  951, 1346],
       [1968,  725, 1485, ...,   33,   62,   29]])

In [58]:
np.sum(seqs == 2)

92

In [61]:
dataset.model.lg.vocsize

2971

In [ ]:
def fucking_sample(state, indexes):
    graphs, finished = state
    batch_len = len(indexes)
    logits = logits[:,self.nspecs:]
    current = [None] * batch_len
    for i in range(len(graphs)):
        graph = graphs[i]
        if graph and (not finished[i]):
            mask = self.get_compat_rules(graph)
            if mask.sum() == 0.:
                finished[i] = True
                current[i] = placeholder_idx
                continue

            logits_of_a_rule = logits[i] * mask
            bool_mask = mask.astype(np.bool)
            prob = softmax(logits_of_a_rule[bool_mask])
            current[i] = np.random.choice(np.array(range(logits.shape[-1]))[bool_mask], 1, p=prob)[0]
            if not check_compat(graphs[i], self.lg.rules[current[i]]):
                raise Exception(
                    "Failed to combine rules! {} and {}".format(graphs[i], self.lg.rules[current[i]]))
            graphs[i] = combine_rules(graphs[i], self.lg.rules[current[i]])

        elif (not graph):
            prob = softmax(logits[i])
            current[i] = np.random.choice(range(logits.shape[-1]), 1, p=prob)[0]
            graphs[i] = self.lg.rules[current[i]]
        else:
            current[i] = placeholder_idx
    state = (graphs, finished)
    return state, [c for c in current]

In [ ]:
state = dataset.model.init_state(100)
for step in range(140):
    state = dataset.model.sample(state, seqs[])
    
    
    
        graphs, finished = state
    batch_len = logits.shape[0]
    logits = logits[:,self.nspecs:]
    current = [None] * batch_len
    for i in range(len(graphs)):
        graph = graphs[i]
        if graph and (not finished[i]):
            mask = self.get_compat_rules(graph)
            if mask.sum() == 0.:
                finished[i] = True
                current[i] = placeholder_idx
                continue

            logits_of_a_rule = logits[i] * mask
            bool_mask = mask.astype(np.bool)
            prob = softmax(logits_of_a_rule[bool_mask])
            current[i] = np.random.choice(np.array(range(logits.shape[-1]))[bool_mask], 1, p=prob)[0]
            if not check_compat(graphs[i], self.lg.rules[current[i]]):
                raise Exception(
                    "Failed to combine rules! {} and {}".format(graphs[i], self.lg.rules[current[i]]))
            graphs[i] = combine_rules(graphs[i], self.lg.rules[current[i]])

        elif (not graph):
            prob = softmax(logits[i])
            current[i] = np.random.choice(range(logits.shape[-1]), 1, p=prob)[0]
            graphs[i] = self.lg.rules[current[i]]
        else:
            current[i] = placeholder_idx
    state = (graphs, finished)
    return state, [c for c in current]

In [ ]:
def sample(self, batch_size,lexical_model, max_length=140):

    start_token = Variable(torch.zeros(batch_size).long())
    start_token[:] = 1 #FixMe -- possible mistake ???

    x = start_token.cuda().unsqueeze(1)

    finished = torch.zeros(batch_size).byte()
    state = lexical_model.init_state(batch_size)
    hidden = self.initHidden(batch_size)
    loss_f = CrossEntropyLoss(reduction='none')
    #hidden = self.endp2hidden(endp)
    #out = self.forward(x)
    log_probs = Variable(torch.zeros(batch_size)).cuda()
    tokens = start_token.clone().cuda()
    tokens = tokens.unsqueeze(1)
    #print(tokens.shape)
    for step in range(max_length):
        logits,hidden = self.forward_to_sample(x,hidden)
        state,x = lexical_model.sample(state,logits.squeeze(2).cpu().detach().numpy(),0)

        x = Variable(torch.cuda.LongTensor(x)).unsqueeze(1)
        tokens = torch.cat([tokens, x], dim=1)
        #print(tokens.shape)
        #print(x.shape)

        #print(logits.shape)
        ##print(x.shape)
        #print(x.is_cuda)
        #print(logits.is_cuda)
        log_probs += loss_f(logits.squeeze(2), x.squeeze(1))
        #rint(x.shape)
    correct = [Chem.MolToSmiles(graph2mol(g)) for g, finished in zip(*state) if finished]
    return correct, log_probs, tokens

In [63]:
logits_agent= torch.load("logits_agent.pk")

In [64]:
logits_agent.shape

torch.Size([100, 2974, 140])

In [70]:
from rdkit import Chem

In [71]:
from legogram import graph2mol

In [80]:
state = dataset.model.init_state(100)
for step in range(140):
    state, _ = dataset.model.sample(state,logits_agent[:, :, step].cpu().detach().numpy(),0)
correct = [Chem.MolToSmiles(graph2mol(g)) if finished else None for g, finished in zip(*state) ]
    
    

In [192]:
smiles = torch.load("correct_smiles.pk")

In [193]:
len(smiles)

100

['Nc1cccc(F)c1',
 'C[C@H](C1CCCC1)C1CCC1',
 'Cc1[nH]cc(=NS)o1',
 'C[C@@H]1COOOO1',
 'CC[O-]',
 'CC1[NH2+]C2(CC(=O)CS1)c1cccc(=O)c1[NH2+]C2(ON1CCCCC1)c1cccc(=S)c1=S',
 'C=C1C(=[NH+][C@H](C)CC)[C@H]2C(=O)N=C(C=Nc3c4c(cc5c(=O)ccc35)C(C)(C(C)C)C(O)(CC)O4)[NH+]12',
 '[N-]=c1oc(=S)c(=O)c2c1[C@]1(OOC(=O)c3ccc(=O)c(-c4occcc4=O)c31)C(=[N+]1O[C@H]3C=CC(=O)C31C=Cc1ccccc1)C2=O',
 'O=S',
 'C[N+]1=NCCC12OOC[C@@]1(C[C@@]13CO3)N([O-])N2O',
 'CC=[NH+]c1cccc2csccc12',
 'CN(c1ccc2c(c1)SCC2)C1CCCCC1',
 'Cc1c2cc(=O)cc(=S)c2c2cc(=[N+]([O-])[C@H]3CCc4cc(Oc5ccc6n(c5)OCCO6)ccc4C3)csc12',
 'CC[O-]',
 'C[S@@+]([NH2+][N-][C@]12CC(OCl)[C@H](CO1)S2)n1ccc2c(cc3ccc4c(c32)N(OOc2cccco2)COC4)c1',
 'Cc1cn[nH]no1',
 'CC1=[N+](c2c(-c3cc([O-])cc(=O)c3)cscc2-c2cccc2=O)CN2C(C1)CC1CCc3c(c(=S)c4cc(=Cc5ccccc5)cc(=[N+](F)c5cccc5)c34)[NH+]12',
 'CC[O-]',
 'Cc1snc2c(=S)ccc3cccc1N32',
 'Oc1coo[nH]o1',
 'C[O-]',
 'C[S@+](C[O-])C1OO1',
 'C[C@@H]1CSSOc2ccncc21',
 'Cc1sccc2c1CCC21C=Cc2oc[nH]cc2C12CC21OO1',
 'Cc1ccc2c(c1)cc1cc3c(CO)cc(=S

In [78]:
len(state[0])

100

In [87]:
len(smiles)

99

In [194]:
import rdkit
from rdkit.Chem.Crippen import MolLogP, MolMR
from rdkit.Chem.Lipinski import  NumHAcceptors, NumHDonors
from rdkit.Chem.rdMolDescriptors import CalcNumRings, CalcNumRotatableBonds, CalcExactMolWt
from providers import robust_standardizer
import requests
from rdkit import Chem
import torch
from copy import deepcopy
import yaml
from sklearn.preprocessing import StandardScaler
import pandas  as pd
from providers import MolecularNotationDataset, SpecialTokenWrapperModel, SmilesModel, robust_standardizer
from legogram import LegoGram
from legogram.apps import LegoGramRNNSampler
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm

In [152]:
import requests
#smiles = ["c1ccccc1", "Br"]
vals_another = requests.post("https://backend.syntelly.com/tempSmilesArrToPredict",
                             json={'smiles': smiles}).json()

In [197]:
for idx in range(len(smiles)):
    if smiles[idx] is not None:
        if Chem.MolFromSmiles(smiles[idx]) is None:
            smiles[idx] = None

In [205]:
smiles2id ={smiles[idx]:idx for idx in range(len(smiles)) if smiles[idx] is not None}

In [212]:
smiles2id

{'O=C1OCC2(CC2)c2c1cn1ccccc21': 3,
 'Cc1cc(-c2ccccc2)cc2c1N(NCF)O[C@H]1CC=N[C@@H]21': 4,
 'C[N+]12C[NH2+][C@@H](CN1C[n+]1cccc(=S)c1=S)OC2': 6,
 'C[C@@H]1C[C@]12N1CC3CN2N[C@@H](C1)N3': 7,
 'O=C1S(c2ccccc2)=[N+]2N=CC=[N+]2OC1(Cl)Cl': 9,
 'Cc1cc2c(c1=O)SC(C1C=C1)=[NH+]O2': 10,
 'CC([O-])[O-]': 11,
 'CBr': 50,
 'CC[O-]': 85,
 'CC1[C@@H]2CCOCC[C@@H]2CC[C@@]12C=[N+](OC1OO1)c1ccccc12': 15,
 'C[C@@H]1CC[C@@H]2CC[C@@H](C1)C2': 18,
 'CC1=NCOCc2c1ccc1ccccc21': 21,
 'Cc1ccccc1': 22,
 'Cc1cc2cssc2n1': 23,
 'CI': 24,
 'C[C@H]1COC2OC3C=CC(OC3)[C@H]21': 25,
 'CC(=c1nnss1)C1CC1': 29,
 'C[C@@H]1Cc2nc3cccoc3c2CO1': 34,
 'N#[N+][C@@H]1C2C=C[C@@H](C2)C1(F)c1ccc2c(c1)CCCC2': 36,
 'C[O-]': 38,
 'C[N+](=C1CC2[C@H](S1)[C@H]2c1o[nH][n+]1C)c1ccccc1': 39,
 'CF': 40,
 'C[C@H]1N[C@@H]2OOS[C@H]2N2CCC3=NCN=NN3[C@H]12': 42,
 'C[C@H]1CCCCc2cccc(-c3ccc4cccc5c4c3CC5)c21': 44,
 'C[C@H]1Oc2c(ccc(=S)c2=[N+]([O-])F)C2C=CO[C@@H]21': 47,
 'N#C[O-]': 49,
 'CC1=NNc2ccccc2N1': 51,
 'Cc1ooc2o[nH]c(=[N+](C3COOC3)[NH+]3CCC3(CCCCCc3c

In [213]:
smiles2id

{'O=C1OCC2(CC2)c2c1cn1ccccc21': 3,
 'Cc1cc(-c2ccccc2)cc2c1N(NCF)O[C@H]1CC=N[C@@H]21': 4,
 'C[N+]12C[NH2+][C@@H](CN1C[n+]1cccc(=S)c1=S)OC2': 6,
 'C[C@@H]1C[C@]12N1CC3CN2N[C@@H](C1)N3': 7,
 'O=C1S(c2ccccc2)=[N+]2N=CC=[N+]2OC1(Cl)Cl': 9,
 'Cc1cc2c(c1=O)SC(C1C=C1)=[NH+]O2': 10,
 'CC([O-])[O-]': 11,
 'CBr': 50,
 'CC[O-]': 85,
 'CC1[C@@H]2CCOCC[C@@H]2CC[C@@]12C=[N+](OC1OO1)c1ccccc12': 15,
 'C[C@@H]1CC[C@@H]2CC[C@@H](C1)C2': 18,
 'CC1=NCOCc2c1ccc1ccccc21': 21,
 'Cc1ccccc1': 22,
 'Cc1cc2cssc2n1': 23,
 'CI': 24,
 'C[C@H]1COC2OC3C=CC(OC3)[C@H]21': 25,
 'CC(=c1nnss1)C1CC1': 29,
 'C[C@@H]1Cc2nc3cccoc3c2CO1': 34,
 'N#[N+][C@@H]1C2C=C[C@@H](C2)C1(F)c1ccc2c(c1)CCCC2': 36,
 'C[O-]': 38,
 'C[N+](=C1CC2[C@H](S1)[C@H]2c1o[nH][n+]1C)c1ccccc1': 39,
 'CF': 40,
 'C[C@H]1N[C@@H]2OOS[C@H]2N2CCC3=NCN=NN3[C@H]12': 42,
 'C[C@H]1CCCCc2cccc(-c3ccc4cccc5c4c3CC5)c21': 44,
 'C[C@H]1Oc2c(ccc(=S)c2=[N+]([O-])F)C2C=CO[C@@H]21': 47,
 'N#C[O-]': 49,
 'CC1=NNc2ccccc2N1': 51,
 'Cc1ooc2o[nH]c(=[N+](C3COOC3)[NH+]3CCC3(CCCCCc3c

In [214]:
smiles = list(filter(lambda x: x is not None, smiles))

In [215]:
smiles

['O=C1OCC2(CC2)c2c1cn1ccccc21',
 'Cc1cc(-c2ccccc2)cc2c1N(NCF)O[C@H]1CC=N[C@@H]21',
 'C[N+]12C[NH2+][C@@H](CN1C[n+]1cccc(=S)c1=S)OC2',
 'C[C@@H]1C[C@]12N1CC3CN2N[C@@H](C1)N3',
 'O=C1S(c2ccccc2)=[N+]2N=CC=[N+]2OC1(Cl)Cl',
 'Cc1cc2c(c1=O)SC(C1C=C1)=[NH+]O2',
 'CC([O-])[O-]',
 'CBr',
 'CC[O-]',
 'CC1[C@@H]2CCOCC[C@@H]2CC[C@@]12C=[N+](OC1OO1)c1ccccc12',
 'C[C@@H]1CC[C@@H]2CC[C@@H](C1)C2',
 'CC1=NCOCc2c1ccc1ccccc21',
 'Cc1ccccc1',
 'Cc1cc2cssc2n1',
 'CI',
 'C[C@H]1COC2OC3C=CC(OC3)[C@H]21',
 'CC(=c1nnss1)C1CC1',
 'CBr',
 'C[C@@H]1Cc2nc3cccoc3c2CO1',
 'N#[N+][C@@H]1C2C=C[C@@H](C2)C1(F)c1ccc2c(c1)CCCC2',
 'C[O-]',
 'C[N+](=C1CC2[C@H](S1)[C@H]2c1o[nH][n+]1C)c1ccccc1',
 'CF',
 'C[C@H]1N[C@@H]2OOS[C@H]2N2CCC3=NCN=NN3[C@H]12',
 'C[C@H]1CCCCc2cccc(-c3ccc4cccc5c4c3CC5)c21',
 'C[C@H]1Oc2c(ccc(=S)c2=[N+]([O-])F)C2C=CO[C@@H]21',
 'N#C[O-]',
 'CBr',
 'CC1=NNc2ccccc2N1',
 'Cc1ooc2o[nH]c(=[N+](C3COOC3)[NH+]3CCC3(CCCCCc3cccc([C@@H]4COC=CO4)c3C3CC3)CC[C@H]3COCCO3)c2c1-c1ccccc1',
 'CC',
 'O=c1ccc2nnnn[n+]2s1'

In [221]:
def scoring_func(smiles, loyality):
    endpoints = requests.get("https://backend.syntelly.com/endpoints").json()
    endpoints_id2name = dict(zip([e['id'] for e in endpoints], [e['view'] for e in endpoints]))
    vals_another = requests.post("https://backend.syntelly.com/tempSmilesArrToPredict",
                         json={'smiles': smiles}).json()
    
    for idx in range(len(vals_another)):
        elem = vals_another[idx]['data']
        for e in elem:
            e["endpoint_id"] = endpoints_id2name[e["endpoint_id"]]
    e2v = []
    for idx in range(len(vals_another)):
        e2v.append(dict(zip([e['endpoint_id'] for e in vals_another[idx]['data']], [e['value'] for e in vals_another[idx]['data']])))
        
    smiles = [val['smiles'] for val in vals_another]
    mols = [robust_standardizer(mol) for mol in smiles]
    mols = [Chem.MolFromSmiles(mol) for mol in mols]
    molecular_weights = [CalcExactMolWt(mol) for mol in mols]
    logp = [MolLogP(mol) for mol in mols]
    atom_count = [mol.GetNumAtoms() for mol in mols]
    molar_reflactivity = [MolMR(mol) for mol in mols]
    numRings = [CalcNumRings(mol) for mol in mols]
    numRotBonds = [CalcNumRotatableBonds(mol) for mol in mols]
    numHAcceptors = [NumHAcceptors(mol) for mol in mols]
    numHDonors = [NumHDonors(mol) for mol in mols]
    bcf = [e['Bioconcentration factor'] for e in e2v]
    dev_tox = [e['Developmental toxicity'] for e in e2v]
    flash_point = [e['Flash point'] for e in e2v]
    boiling_point = [e['Boiling point'] for e in e2v]
    melting_points = [e['Melting point'] for e in e2v]
    water_solubility = [e['Water Solubility'] for e in e2v]

    result = [0]*len(smiles)
    for idx in range(len(smiles)):
        val = 0
        if(molecular_weights[idx]<=480 and molecular_weights[idx]>=160):
            val+=1
        if(logp[idx]<=5.6 and logp[idx]>=-0.4):
            val+=1
        if(atom_count[idx]<=70 and atom_count[idx]>=20):
            val+=1
        if(molar_reflactivity[idx]>=40 and molar_reflactivity[idx]<=130):
            val+=1
        if(bcf[idx]<3):
            val+=1
        if(dev_tox[idx]=='Negative'):
            val+=1
        if(flash_point[idx]>(350-273.15)):
            val+=1
        if(boiling_point[idx]>(300-273.15)):
            val+=1
        if(numRings[idx]>0):
            val+=1
        if(numRotBonds[idx]<5):
            val+=1
        if(numHAcceptors[idx]<=10):
            val+=1
        if(numHDonors[idx]<=5):
            val+=1
        if(melting_points[idx]>140):
            val+=10

        if(val>=loyality):
            result[idx] = val
        return list(zip(smiles, result))

    
    

In [222]:
res = scoring_func(smiles, 12)

In [220]:
val = 

[('O=C1OCC2(CC2)c2c1cn1ccccc21', 20),
 ('Cc1cc(-c2ccccc2)cc2c1N(NCF)O[C@H]1CC=N[C@@H]21', 0),
 ('C[N+]12C[NH2+][C@@H](CN1C[n+]1cccc(=S)c1=S)OC2', 0),
 ('C[C@@H]1C[C@]12N1CC3CN2N[C@@H](C1)N3', 0),
 ('O=C1S(c2ccccc2)=[N+]2N=CC=[N+]2OC1(Cl)Cl', 0),
 ('Cc1cc2c(c1=O)SC(C1C=C1)=[NH+]O2', 0),
 ('CC([O-])[O-]', 0),
 ('CBr', 0),
 ('CC[O-]', 0),
 ('CC1[C@@H]2CCOCC[C@@H]2CC[C@@]12C=[N+](OC1OO1)c1ccccc12', 0),
 ('C[C@@H]1CC[C@@H]2CC[C@@H](C1)C2', 0),
 ('CC1=NCOCc2c1ccc1ccccc21', 0),
 ('Cc1ccccc1', 0),
 ('Cc1cc2cssc2n1', 0),
 ('CI', 0),
 ('C[C@H]1COC2OC3C=CC(OC3)[C@H]21', 0),
 ('CC(=c1nnss1)C1CC1', 0),
 ('CBr', 0),
 ('C[C@@H]1Cc2nc3cccoc3c2CO1', 0),
 ('N#[N+][C@@H]1C2C=C[C@@H](C2)C1(F)c1ccc2c(c1)CCCC2', 0),
 ('C[O-]', 0),
 ('C[N+](=C1CC2[C@H](S1)[C@H]2c1o[nH][n+]1C)c1ccccc1', 0),
 ('CF', 0),
 ('C[C@H]1N[C@@H]2OOS[C@H]2N2CCC3=NCN=NN3[C@H]12', 0),
 ('C[C@H]1CCCCc2cccc(-c3ccc4cccc5c4c3CC5)c21', 0),
 ('C[C@H]1Oc2c(ccc(=S)c2=[N+]([O-])F)C2C=CO[C@@H]21', 0),
 ('N#C[O-]', 0),
 ('CBr', 0),
 ('CC1=NNc2ccccc

In [227]:
val = [0]*100

In [228]:
for elem in res:
    val[smiles2id[elem[0]]] = elem[1]

In [153]:
len(smiles)

58

In [144]:
mol = Chem.MolFromSmiles("Cc1[nH]cc(=NS)o1")

In [146]:
mol is None

True

In [155]:
smiles

['Nc1cccc(F)c1',
 'C[C@H](C1CCCC1)C1CCC1',
 'C[C@@H]1COOOO1',
 'CC[O-]',
 'C=C1C(=[NH+][C@H](C)CC)[C@H]2C(=O)N=C(C=Nc3c4c(cc5c(=O)ccc35)C(C)(C(C)C)C(O)(CC)O4)[NH+]12',
 'O=S',
 'C[N+]1=NCCC12OOC[C@@]1(C[C@@]13CO3)N([O-])N2O',
 'CN(c1ccc2c(c1)SCC2)C1CCCCC1',
 'CC[O-]',
 'CC[O-]',
 'Cc1snc2c(=S)ccc3cccc1N32',
 'Oc1coo[nH]o1',
 'C[O-]',
 'C[S@+](C[O-])C1OO1',
 'C[C@@H]1CSSOc2ccncc21',
 'Cc1ccc2c(c1)cc1cc3c(CO)cc(=S)cc3cc12',
 'Cc1ccc2c(c1)[NH2+]C[C@H]1Cc3cccc(C(C)(COCOF)C4CCCC4)c3[C@H]21',
 'C[C@H]1[NH2+][C@@H]2N=[NH+][C@H]1O2',
 'CC[O-]',
 'CN1N=[N+]2CNC=[N+]2[N+](c2c(O)cccc2F)=C1[O-]',
 'Cc1cc2c(o1)OOC[C@@]21O[C@@H](C([O-])(Cl)C([O-])(F)C#N)OOO[C@@H]1c1ccc(=O)[n-]c1',
 'CC1=N[C@]2(OCC(C)c3ncc(=S)c32)OO1',
 'O=C1N=N[C@@H]2C1C=C([O-])[C@@H]2N1CN1',
 'CC1CC=C2CCCC[C@@H](P3([O-])=CCCCOO3)[C@H]21',
 'Br[C@@H]1CP2(c3ccccc3)=C(CO1)OCO2',
 'C[C@@H]1C2C[C@]3(C=N2)[C@H]1CCC31CCCC2=c3ccccc3=N[C@@H]21',
 'CN1NN2C[C@@H](N[S+]2NN2OC[C@@H](NF)O2)P12(NNN1O[NH+]3OO[NH+]1N=[N+]3[O-])N[NH+](O)OO2',
 '[O-]

{58: 'Density', 59: '40 hour Tetrahymena pyriformis IGC50', 61: 'Viscosity', 63: 'Water solubility', 64: 'Surface tension', 65: 'Vapor pressure', 66: '96 hour fathead minnow LC50', 68: ' Thermal conductivity', 35: 'Boiling Point (std.)', 67: 'Melting point', 70: 'LogP octanol-water', 73: 'CYP450', 74: 'CYP1A2', 75: 'CYP2C19', 76: 'CYP2C9', 77: 'CYP2D6', 78: 'CYP3A4', 62: '48 hour Daphnia magna LC50', 60: 'Ames test', 71: 'Soluble in DMSO', 79: '250KDummyAllZeros', 80: 'Retention time', 37: None, 36: None, 39: 'Super Test', 38: 'Water Solubility', 40: 'Vapor Pressure', 50: None, 53: 'Boiling point', 54: 'Flash point', 55: 'Bioconcentration factor', 57: 'Developmental toxicity'}


In [176]:

for idx in range(len(vals_another)):
    elem = vals_another[idx]['data']
    for e in elem:
        e["endpoint_id"] = endpoints_id2name[e["endpoint_id"]]
e2v = []
for idx in range(len(vals_another)):
    e2v.append(dict(zip([e['endpoint_id'] for e in vals_another[idx]['data']], [e['value'] for e in vals_another[idx]['data']])))

In [177]:
vals_another[0]['data']

[{'endpoint_id': 'Water Solubility', 'units': 'LogS', 'value': -0.7},
 {'endpoint_id': 'Vapor Pressure', 'units': 'log(mmHg)', 'value': -0.6},
 {'endpoint_id': 'Boiling point', 'units': 'Celsius', 'value': 187},
 {'endpoint_id': 'Flash point', 'units': 'Celsius', 'value': 70},
 {'endpoint_id': 'Bioconcentration factor',
  'units': 'Log10(BCF)',
  'value': 0.8},
 {'endpoint_id': 'Density', 'units': 'g/cm3', 'value': 1.16},
 {'endpoint_id': '40 hour Tetrahymena pyriformis IGC50',
  'units': 'Log10 (IGC50 mol/L)',
  'value': 2.9},
 {'endpoint_id': 'Developmental toxicity', 'units': '', 'value': 'Negative'},
 {'endpoint_id': 'Ames test', 'units': '', 'value': 'Negative'},
 {'endpoint_id': 'Viscosity', 'units': 'log10(viscosity cP)', 'value': 0.5},
 {'endpoint_id': '48 hour Daphnia magna LC50',
  'units': 'Log10 (LC50 mol/L)',
  'value': 4},
 {'endpoint_id': 'Melting point', 'units': 'Celsius', 'value': 0},
 {'endpoint_id': 'CYP1A2', 'units': '', 'value': 'Noninhibitor'},
 {'endpoint_id': '

In [189]:
#def scoring_func(smiles):
loyality = 12
smiles = [val['smiles'] for val in vals_another]
mols = [robust_standardizer(mol) for mol in smiles]
mols = [Chem.MolFromSmiles(mol) for mol in mols]
molecular_weights = [CalcExactMolWt(mol) for mol in mols]
logp = [MolLogP(mol) for mol in mols]
atom_count = [mol.GetNumAtoms() for mol in mols]
molar_reflactivity = [MolMR(mol) for mol in mols]
numRings = [CalcNumRings(mol) for mol in mols]
numRotBonds = [CalcNumRotatableBonds(mol) for mol in mols]
numHAcceptors = [NumHAcceptors(mol) for mol in mols]
numHDonors = [NumHDonors(mol) for mol in mols]
bcf = [e['Bioconcentration factor'] for e in e2v]
dev_tox = [e['Developmental toxicity'] for e in e2v]
flash_point = [e['Flash point'] for e in e2v]
boiling_point = [e['Boiling point'] for e in e2v]
melting_points = [e['Melting point'] for e in e2v]
water_solubility = [e['Water Solubility'] for e in e2v]

result = [0]*len(smiles)
for idx in range(len(smiles)):
    val = 0
    if(molecular_weights[idx]<=480 and molecular_weights[idx]>=160):
        val+=1
    if(logp[idx]<=5.6 and logp[idx]>=-0.4):
        val+=1
    if(atom_count[idx]<=70 and atom_count[idx]>=20):
        val+=1
    if(molar_reflactivity[idx]>=40 and molar_reflactivity[idx]<=130):
        val+=1
    if(bcf[idx]<3):
        val+=1
    if(dev_tox[idx]=='Negative'):
        val+=1
    if(flash_point[idx]>(350-273.15)):
        val+=1
    if(boiling_point[idx]>(300-273.15)):
        val+=1
    if(numRings[idx]>0):
        val+=1
    if(numRotBonds[idx]<5):
        val+=1
    if(numHAcceptors[idx]<=10):
        val+=1
    if(numHDonors[idx]<=5):
        val+=1
    if(melting_points[idx]>140):
        val+=10

    if(val>=loyality):
        result[idx] = val



In [191]:
result

[0,
 0,
 0,
 0,
 18,
 0,
 19,
 0,
 0,
 0,
 20,
 19,
 0,
 0,
 0,
 21,
 0,
 17,
 0,
 19,
 19,
 20,
 18,
 0,
 0,
 21,
 16,
 0,
 19,
 20,
 20,
 0,
 18,
 21,
 21,
 20,
 0,
 0,
 21,
 18,
 0,
 20,
 0,
 20,
 21,
 20,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 21,
 21,
 0]

In [182]:
res = scoring_func(smiles)

{58: 'Density', 59: '40 hour Tetrahymena pyriformis IGC50', 61: 'Viscosity', 63: 'Water solubility', 64: 'Surface tension', 65: 'Vapor pressure', 66: '96 hour fathead minnow LC50', 68: ' Thermal conductivity', 35: 'Boiling Point (std.)', 67: 'Melting point', 70: 'LogP octanol-water', 73: 'CYP450', 74: 'CYP1A2', 75: 'CYP2C19', 76: 'CYP2C9', 77: 'CYP2D6', 78: 'CYP3A4', 62: '48 hour Daphnia magna LC50', 60: 'Ames test', 71: 'Soluble in DMSO', 79: '250KDummyAllZeros', 80: 'Retention time', 37: None, 36: None, 39: 'Super Test', 38: 'Water Solubility', 40: 'Vapor Pressure', 50: None, 53: 'Boiling point', 54: 'Flash point', 55: 'Bioconcentration factor', 57: 'Developmental toxicity'}


KeyError: 'Bioconcentration factor'

In [ ]:
print(f"round {round}")
number_generate = 100
endp = torch.tensor(scaler.transform(np.array([conditions[:-1]])))
print(endp.shape)

c = deepcopy(endp)
c = [str(l) for l in list(c.numpy())]
#endp = endp.unsqueeze(0)
endp = endp.repeat(100,1)
endp = endp.unsqueeze(0)
endp = endp.repeat(3, 1, 1)

endp = endp.float()
endp = endp.cuda()
res = model.sample(endp, number_generate, dataset.model)
valid = len(res) * 100 / number_generate
print("valid : {} %".format(valid))
#writer.add_scalar("Valid", valid, cnt)
res = [robust_standardizer(mol) for mol in res]
res = list(filter(lambda x: x is not None, res))
mols  =res
print("Mols obtained")
print(mols)
vals_another = requests.post("https://backend.syntelly.com/tempSmilesArrToPredict",
                             json={'smiles': mols}).json()
for idx in range(len(vals_another)):
    elem = vals_another[idx]['data']
    for e in elem:
        e["endpoint_id"] = endpoints_id2name[e["endpoint_id"]]
e2v = []
for idx in range(len(vals_another)):
    e2v.append(dict(zip([e['endpoint_id'] for e in vals_another[idx]['data']], [e['value'] for e in vals_another[idx]['data']])))
smiles = [val['smiles'] for val in vals_another]
mols = [robust_standardizer(mol) for mol in smiles]
mols = [Chem.MolFromSmiles(mol) for mol in mols]
molecular_weights = [CalcExactMolWt(mol) for mol in mols]
logp = [MolLogP(mol) for mol in mols]
atom_count = [mol.GetNumAtoms() for mol in mols]
molar_reflactivity = [MolMR(mol) for mol in mols]
numRings = [CalcNumRings(mol) for mol in mols]
numRotBonds = [CalcNumRotatableBonds(mol) for mol in mols]
numHAcceptors = [NumHAcceptors(mol) for mol in mols]
numHDonors = [NumHDonors(mol) for mol in mols]
bcf = [e['Bioconcentration factor'] for e in e2v]
dev_tox = [e['Developmental toxicity'] for e in e2v]
flash_point = [e['Flash point'] for e in e2v]
boiling_point = [e['Boiling point'] for e in e2v]
melting_points = [e['Melting point'] for e in e2v]
water_solubility = [e['Water Solubility'] for e in e2v]

result = [0]*len(smiles)
for idx in range(len(smiles)):
    val = 0
    if(molecular_weights[idx]<=480 and molecular_weights[idx]>=160):
        val+=1
    if(logp[idx]<=5.6 and logp[idx]>=-0.4):
        val+=1
    if(atom_count[idx]<=70 and atom_count[idx]>=20):
        val+=1
    if(molar_reflactivity[idx]>=40 and molar_reflactivity[idx]<=130):
        val+=1
    if(bcf[idx]<3):
        val+=1
    if(dev_tox[idx]=='Negative'):
        val+=1
    if(flash_point[idx]>(350-273.15)):
        val+=1
    if(boiling_point[idx]>(300-273.15)):
        val+=1
    if(numRings[idx]>0):
        val+=1
    if(numRotBonds[idx]<5):
        val+=1
    if(numHAcceptors[idx]<=10):
        val+=1
    if(numHDonors[idx]<=5):
        val+=1

    if(val/12>=loyality):
        result[idx] = val

print(result)
for idx in range(len(result)):
    if (result[idx]>0):
        result_arr.append((smiles[idx], result[idx], (melting_points[idx], boiling_point[idx], water_solubility[idx]), mean_squared_error(
            scaler.transform(np.array([[melting_points[idx], boiling_point[idx], water_solubility[idx]]])), scaler.transform(np.array([conditions[:-1]]))
        )))

result_arr.sort(key = lambda x: x[3])

print(result_arr[:10])


In [40]:
dataset.model.decode(elem)

IndexError: list index out of range

In [6]:
dataset.model.decode()

2974